Import Statements

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from dotenv import load_dotenv
load_dotenv()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.config import TRANSFORMED_DATA_DIR
from src.data_utils import split_time_series_data
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from src.plot_utils import plot_aggregated_time_series

Data Loading and Test Train split

In [2]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df.head(5)

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,target
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2023-01-29,2,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2023-01-30,2,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2023-01-31,2,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2023-02-01,2,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2023-02-02,2,0


(55900, 674)
(55900,)
(31720, 674)
(31720,)


Baseline Model using Previous Hour

In [3]:
class BaselineModelPreviousHour:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]
    

model = BaselineModelPreviousHour()
predictions = model.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")
mlflow = set_mlflow_tracking()
log_model_to_mlflow(model, X_test, "BaselineModelPreviousHour", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:MLflow tracking URI and credentials set.


6.7109


INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 6.7109394703657
c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500P1\CDA500P1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
c:\U

🏃 View run skittish-cow-525 at: https://dagshub.com/singhvarunnn789/CDA500P1.mlflow/#/experiments/0/runs/8cac7d7e9d924a988d92dbd321410f90
🧪 View experiment at: https://dagshub.com/singhvarunnn789/CDA500P1.mlflow/#/experiments/0


Baseline Model using the past week (7 days * 24 hours)

In [4]:
class BaselineModelPreviousWeek:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*24}"]

model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek


5.2301


INFO:src.experiment_utils:Logged mean_absolute_error: 5.230075662042875
c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500P1\CDA500P1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineModelPreviousWeek' already exists. Creating a n

🏃 View run nebulous-ant-665 at: https://dagshub.com/singhvarunnn789/CDA500P1.mlflow/#/experiments/1/runs/1750341a10a04e77ac7b2ff6582791c7
🧪 View experiment at: https://dagshub.com/singhvarunnn789/CDA500P1.mlflow/#/experiments/1


Baseline Model using Past 4 Weeks (4 Weeks * 7 * 24 split from 0-7-14-21-28)

In [5]:
class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        """
        The fit method is not used in this baseline model as it does not learn
        from the training data.
        """
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Predicts the average of the last 4 weeks (28 days) for each test instance.

        Parameters:
            X_test (pd.DataFrame): The test DataFrame containing lagged features
                                   (e.g., rides_t-{7*24}, rides_t-{14*24}, etc.).

        Returns:
            np.array: An array of predictions based on the average of the last 4 weeks.
        """
        # Define the columns for the last 4 weeks
        last_4_weeks_columns = [
            f"rides_t-{7*24}",  # 1 week ago
            f"rides_t-{14*24}", # 2 weeks ago
            f"rides_t-{21*24}", # 3 weeks ago
            f"rides_t-{28*24}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")

        # Calculate the average of the last 4 weeks
        predictions = X_test[last_4_weeks_columns].mean(axis=1)

        return predictions.to_numpy()
    
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

X_test[X_test['pickup_location_id']==43].head()
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
5002,12,6,1,1,0,7,8,46,41,48,...,131,74,82,62,64,34,38,12,2023-09-01,43
5003,13,7,1,0,0,4,2,11,28,35,...,122,113,80,107,67,58,39,9,2023-09-02,43
5004,14,5,3,1,0,3,7,13,12,27,...,174,166,174,142,86,63,53,13,2023-09-03,43
5005,2,2,0,0,1,5,13,36,34,39,...,169,144,90,61,45,28,31,7,2023-09-04,43
5006,3,0,0,0,1,4,5,44,61,59,...,136,145,85,46,33,28,9,8,2023-09-05,43


INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks


4.5444


INFO:src.experiment_utils:Logged mean_absolute_error: 4.5443647540983605
c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500P1\CDA500P1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
Registered model 'BaselineModelLast4Weeks' already exists. Creating a ne

🏃 View run magnificent-frog-783 at: https://dagshub.com/singhvarunnn789/CDA500P1.mlflow/#/experiments/2/runs/edfad40ecfeb4883b6efd9a9717f7b15
🧪 View experiment at: https://dagshub.com/singhvarunnn789/CDA500P1.mlflow/#/experiments/2


Plotting prediction from the latest baseline model (4 weeks)

In [6]:
plot_aggregated_time_series(X_test, y_test, 5002, predictions)